# Chargement des données

In [23]:
# mise en place des chemins des fichiers contenant les datasets
dossier_data = "data/brutes/"
nom_fichier_application_train = "application_train.csv"
nom_fichier_application_test = "application_test.csv"
nom_fichier_bureau = "bureau.csv"
nom_fichier_bureau_balance = "bureau_balance.csv"
nom_fichier_POS_CASH_balance = "POS_CASH_balance.csv"
nom_fichier_credit_card_balance = "credit_card_balance.csv"
nom_fichier_previous_application = "previous_application.csv"
nom_fichier_installments_payments = "installments_payments.csv"

In [24]:
import pandas as pd

# chargement des datasets (BDD complète)
application_train = pd.read_csv(dossier_data + nom_fichier_application_train)
application_test = pd.read_csv(dossier_data + nom_fichier_application_test)
bureau = pd.read_csv(dossier_data + nom_fichier_bureau)
bureau_balance = pd.read_csv(dossier_data + nom_fichier_bureau_balance)
POS_CASH_balance = pd.read_csv(dossier_data + nom_fichier_POS_CASH_balance)
credit_card_balance = pd.read_csv(dossier_data + nom_fichier_credit_card_balance)
previous_application = pd.read_csv(dossier_data + nom_fichier_previous_application)
installments_payments = pd.read_csv(dossier_data + nom_fichier_installments_payments)